# 3. Les stations les plus proches avec pandas et une fonction distance

Pour avoir de meilleur résultat qu'au premier essai nous allons raisonner avec des dataframes et un fonction distance entre des points GPS.

In [8]:
import pandas as pd 
import geopandas as gpd
pd.options.mode.chained_assignment = None  # default='warn'
import module_projet as md

On récupère le dataframe instantané et on le nettoie.

In [9]:
dat_inst = gpd.read_file('data/fichier_inst.csv')

In [10]:
data= md.data_carb(dat_inst, 'Gazole',['adresse', 'ville', 'prix_valeur', 'prix_nom', 'geom'])

On fait une onction pour calculer la distance en métre à partir de données de longitude et de latitude.

In [11]:
from math import sin, cos, acos, pi
 
def dms2dd(d, m, s):
    """Convertit un angle "degrés minutes secondes" en "degrés décimaux"
    """
    return d + m/60 + s/3600
 
def dd2dms(dd):
    """Convertit un angle "degrés décimaux" en "degrés minutes secondes"
    """
    d = int(dd)
    x = (dd-d)*60
    m = int(x)
    s = (x-m)*60
    return d, m, s
 
def deg2rad(dd):
    """Convertit un angle "degrés décimaux" en "radians"
    """
    return dd/180*pi
 

def rad2deg(rd):
    """Convertit un angle "radians" en "degrés décimaux"
    """
    return rd/pi*180
 
#############################################################################
def distanceGPS(p1, p2):
    """Retourne la distance en mètres entre les 2 points A et B connus grâce à
       leurs coordonnées GPS (en radians).
    """
    latA, longA = p1
    latB, longB = p2
    # Rayon de la terre en mètres (sphère IAG-GRS80)
    RT = 6378137
    # angle en radians entre les 2 points
    S = acos(sin(latA)*sin(latB) + cos(latA)*cos(latB)*cos(abs(longB-longA)))
    # distance entre les 2 points, comptée sur un arc de grand cercle
    return S*RT




On fait comme précedemment une fonction pour trouver les stations les 10 stations les plus proches. 

In [12]:
def dix_plus_proche_card(df, posi, carb):
    data_1 = md.data_carb(df, carb, ['adresse', 'ville', 'geom', 'prix_valeur', 'prix_nom'])
    pos = list(map(float, md.Adresse_to_gps(posi).split(',')))
    def distancetoposi(p2):
        pos_carb = list(map(float, p2.split(',')))
        return distanceGPS(pos, pos_carb)
    new_df = data_1['geom'].apply(distancetoposi).sort_values().head(10)
    return data_1.loc[new_df.index.to_list()]

In [13]:
dix_plus_proche_card(dat_inst, "1 rue de Rivoli, Paris, France" , 'SP95')

,adresse,ville,geom,prix_valeur,prix_nom
11862,175 Avenue Ledru-Rollin,PARIS,"48.85667,2.3791",2.9,SP95
13891,"8,10,10bis Rue Bailleul",PARIS,"48.861,2.341",2.24,SP95
880,15 Avenue Duquesne,PARIS,"48.853,2.308",2.6,SP95
4535,69 Avenue KlÃ©ber,Paris,"48.865,2.289",2.64,SP95
17011,44 Avenue de Paris,VINCENNES,"48.845,2.433",1.919,SP95
30640,173 Avenue Daumesnil,Paris,"48.84074,2.39104",2.199,SP95
30187,253 Boulevard Raspail,PARIS,"48.838,2.331",2.69,SP95
10661,RN 197 LIEU-DIT LES PADULES,CALVI,"42.572,8.759",1.79,SP95
7367,Station TOTAL - relais de la balagne,CALVI,"42.56429,8.75541",1.69,SP95
16896,Avenue Christophe Colomb,Calvi,"42.561,8.755",1.66,SP95


Le résultat est beaucoup plus concluant et l'on obtient bien les dix stations les plus proches de l'usager. On peut cependant noter que le résultat est donnée pour la distance à vol d'oiseau, on pourrait améliorer en faisant selon un itinéraire.